In [9]:
# import preprocess as prep
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 
import importlib
import preprocess_lite
import preprocess
import SLR_model
from scipy. stats import mode
import tensorflow as tf
import numpy as np
# model will output multiple(15) results/sec, how are we gonna handle it?

In [10]:
importlib.reload(SLR_model)
importlib.reload(preprocess_lite)
importlib.reload(preprocess)



<module 'preprocess' from '/home/shd/BomNae-SLR/preprocess.py'>

In [11]:
# Configurations
model_dir = "saves"
THRESHOLD = 0.5
# batch_size = 16
file_name="11-20-16-30epochs-5times_final"

In [12]:
# 세이브된 모델 로드
load_path = os.path.join(model_dir, "ckpt",file_name+".keras")
model = SLR_model.load_model(load_path)

In [13]:
# load data
l_raws=[]
r_raws=[]
p_raws=[]
y_raws=[]
person_id = 8
# TODO hardcode path
# a video
for j in sorted(os.listdir(os.path.join(preprocess_lite.getoutputdir(),str(person_id)))):
    j="0002.npz"
    l_raw, r_raw, p_raw, y_raw ,w= preprocess_lite.load_data(f"{person_id}/{j}")
    
    l_raws.append(l_raw)
    r_raws.append(r_raw)
    p_raws.append(p_raw)
    y_raws.append(y_raw)
    l_test, each = SLR_model.serialize(l_raws)
    r_test, each = SLR_model.serialize(r_raws)
    p_test, each = SLR_model.serialize(p_raws, stride=2,is_pose = True)
    x_test = (l_test, r_test, p_test)

    predictions = model.predict(x_test,32)
    pred=SLR_model.decode_onehot2d(predictions)
    for i in range(len(predictions)):
        print(f"{y_raw}:{pred[i]}")
        if pred[i]==y_raw:
            print(tf.math.top_k(predictions[i],k=2).values.numpy())
            break
            
    # print(SLR_model.decode_onehot2d(predictions))

    break
    

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step
1809:2233
1809:2233
1809:569
1809:569
1809:2233


In [21]:
# load data
l_raws=[]
r_raws=[]
p_raws=[]
y_raws=[]
person_id = 18
count=0
name_list=[]
# TODO hardcode path
# a video
val_list=sorted(os.listdir(os.path.join(preprocess_lite.getoutputdir('val'),str(person_id))))
# val_list=sorted(os.listdir(os.path.join(preprocess_lite.getoutputdir(),str(person_id))))

for j in val_list:
    # j="0001.npz"
    try:
        l_raw, r_raw, p_raw, y_raw = preprocess_lite.load_data(f"{person_id}/{j}",'val')
        # l_raw, r_raw, p_raw, y_raw,w = preprocess_lite.load_data(f"{person_id}/{j}")
        
    except Exception as e:
        print(e)
        continue
    l_raws.append(l_raw)
    r_raws.append(r_raw)
    p_raws.append(p_raw)
    y_raws.append(y_raw)
    l_test, each = SLR_model.serialize(l_raws)
    r_test, each = SLR_model.serialize(r_raws)
    p_test, each = SLR_model.serialize(p_raws, stride=2,is_pose = True)
    x_test = (l_test, r_test, p_test)
    try:
        predictions = model.predict(x_test,32,verbose=0)
        pred=SLR_model.decode_onehot2d(predictions)
        num=mode(pred)
        for i in range(len(predictions)):
            if pred[i]==y_raw:
                top=tf.math.top_k(predictions[i],k=2).values.numpy()
                break
            else: top=np.array([0,0]) 
        ans=(abs(top[0]-top[1]))
    except Exception as e:
        log_msg=f'{j}) {e}\n'
    else:
        log_msg=f'{j}):{ans}\n'
        
        if 0==ans:
            count+=1
            name_list.append(j)
    finally:
        with open(file_name+"_test_log.txt","a") as log:
            # if int(y_raw)!=int(num[0]):
            log.write(log_msg)
            # print(log_msg)
        l_raws.clear()
        r_raws.clear()
        p_raws.clear()
        y_raws.clear()
print(count)
print(len(val_list))
print(count/len(val_list)*100)

2024-11-20 20:20:01.129450: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-11-20 20:20:04.177042: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-11-20 20:20:09.726331: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1105
2554
43.265465935787


In [22]:
print(count)
print(len(name_list))


1105
1105


In [110]:
# # dummy x_raw data
# l_raws = []
# duration_list = []
# h_elem = np.zeros(shape=(4,5,3))
# # p_elem = np.zeros(shape=(3,10))
# # comb_elem = [h_elem, h_elem, p_elem]
# x_elem = []
# duration = (np.random.randint(90,100))
# duration_list.append(duration)
# for j in range(duration): # frame size
#     x_elem.append(h_elem)
#     # x_elem.append(comb_elem)
# l_raws.append(x_elem)
# r_raws = []
# 
# h_elem = np.zeros(shape=(4,5,3))
# # p_elem = np.zeros(shape=(3,10))
# # comb_elem = [h_elem, h_elem, p_elem]
# x_elem = []
# duration = duration_list[0]
# for j in range(duration): # frame size
#     x_elem.append(h_elem)
#     # x_elem.append(comb_elem)
# r_raws.append(x_elem)
# p_raws = []
# 
# # h_elem = np.zeros(shape=(4,5,3))
# p_elem = np.zeros(shape=(3,10))
# # comb_elem = [h_elem, h_elem, p_elem]
# x_elem = []
# duration = duration_list[0]
# for j in range(duration): # frame size
#     # x_elem.append(h_elem)
#     x_elem.append(p_elem)
#     # x_elem.append(comb_elem)
# p_raws.append(x_elem)

In [111]:
l_test, each = SLR_model.serialize(l_raws)
r_test, each = SLR_model.serialize(r_raws)
p_test, each = SLR_model.serialize(p_raws, stride=2,is_pose = True)
x_test = (l_test, r_test, p_test)

# dataset = SLR_model.convert_to_dataset_single(x_test, batch_size)


# with open(hist_path, 'w') as file:
#     json.dump(hist.history, file)

In [8]:
y_raw

2769

In [9]:

predictions = model.predict(x_test,32)
print(SLR_model.decode_onehot2d(predictions))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
tf.Tensor([ 619   62   28 1364   28  619  619  820], shape=(8,), dtype=int64)


In [106]:
predictions

array([[1.06856385e-10, 2.69137300e-19, 6.27669255e-16, ...,
        3.35132165e-11, 4.32728089e-11, 1.56990765e-09],
       [4.79685180e-10, 7.06360674e-20, 8.91828334e-18, ...,
        6.38787079e-11, 2.81286616e-09, 4.90596175e-10],
       [8.79550393e-13, 1.02109870e-19, 1.40288393e-17, ...,
        4.34852722e-14, 1.05808792e-10, 1.22884808e-15],
       ...,
       [1.00203144e-13, 1.65873783e-22, 2.90799827e-24, ...,
        2.71007612e-17, 1.26532996e-12, 5.17534312e-13],
       [1.20203743e-16, 1.76175004e-24, 4.23157955e-29, ...,
        1.74624015e-17, 4.52590236e-14, 9.22390716e-16],
       [4.83049390e-16, 1.69469557e-28, 8.00105026e-30, ...,
        1.01956868e-17, 1.50272268e-14, 1.89650399e-16]], dtype=float32)

In [ ]:
# pred_bin = predictions.round().astype(np.int32)
# pred_bin

In [ ]:
importlib.reload(SLR_model)

In [ ]:
SLR_model.decode_onehot2d(predictions)

In [ ]:
SLR_model.bin2num([1,0,1,1,1,0,1,1,1,0,1,0])

In [ ]:
model.predict